In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
# C:\\Users\\dlals\\Desktop\\project_code\\석적점으로매장시간맞춰보기.xlsx
# C:\\Users\\dlals\\Desktop\\project_code\\시간대별_매출_석적점_전처리완료.xlsx
# C:\\Users\\dlals\\Desktop\\project_code\\
# 매장시간2csv파일

In [2]:
receipt_price = input('1.영수증별 매출내역 파일을 위치와 함께 입력(예시:C:\\test\\example.xlsx):')
time_series_price = input('2.시간대별 매출내역 파일을 위치와 함께 입력(예시:C:\\test\\example.xlsx):')
f_dir = input('3.출력할 경로를 입력(예시:C:\\test):')
keyword = input('4.영수증별 매출파일 출력할 파일명을 입력(예시:example):')
time_keyword = input('5.시간대별 매출파일 출력할 파일명을 입력(예시:example):')

1.영수증별 매출내역 파일을 위치와 함께 입력(예시:C:\test\example.xlsx):C:\\Users\\dlals\\Desktop\\project_code\\신서점_영수증별 메뉴 2020_7월.csv
2.시간대별 매출내역 파일을 위치와 함께 입력(예시:C:\test\example.xlsx):C:\\Users\\dlals\\Desktop\\project_code\\시간대별_매출_석적점_전처리완료.xlsx
3.출력할 경로를 입력(예시:C:\test):C:\\Users\\dlals\\Desktop\\project_code\\
4.영수증별 매출파일 출력할 파일명을 입력(예시:example):성서점연관분석TEST
5.시간대별 매출파일 출력할 파일명을 입력(예시:example):삭제파일


In [32]:
xlsx = pd.read_csv(receipt_price)
#xlsx.dropna(subset=['판매시간'],inplace=True)
xlsx

,Unnamed: 0,판매일자,영수증번호,판매시간,영수증별총매출,현금,카드,key,구분,판매시간.1,메뉴코드,영수증번호.1,메뉴명,메뉴가격,메뉴수량,메뉴가격*수량
0,0,2020-07-01,01-0001,09:58:56,22000,0,22000,r1,판매,09:58:56,Z001,01-0001,돼지국밥,7000,2,14000
1,1,2020-07-01,01-0001,09:58:56,22000,0,22000,r1,판매,09:58:56,Z002,01-0001,순대국밥,8000,1,8000
2,2,2020-07-01,01-0002,10:19:45,36000,0,36000,r2,판매,10:19:45,Z031,01-0002,양념주물럭(1인분),8000,2,16000
3,3,2020-07-01,01-0002,10:19:45,36000,0,36000,r2,판매,10:19:45,Z054,01-0002,짬뽕순두부국밥,8000,1,8000
4,4,2020-07-01,01-0002,10:19:45,36000,0,36000,r2,판매,10:19:45,Z072,01-0002,참소주,4000,3,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,8589,2020-07-31,01-0192,09:44:37,23500,0,23500,r4852,판매,09:44:37,Z002,01-0192,순대국밥,8000,1,8000
8590,8590,2020-07-31,01-0192,09:44:37,23500,0,23500,r4852,판매,09:44:37,Z005,01-0192,모둠국밥,8500,1,8500
8591,8591,2020-07-31,01-0193,09:44:49,14000,14000,0,r4853,판매,09:44:49,Z001,01-0193,돼지국밥,7000,2,14000
8592,8592,2020-07-31,01-0194,09:46:56,32000,0,32000,r4854,판매,09:46:56,Z038,01-0194,무보까정식,10000,2,20000


In [33]:
df_time_price = pd.read_excel(time_series_price)
df_time_price.rename(columns={'Unnamed: 0':'시간'},inplace=True)
df_time_price

,시간,날짜,매출액
0,0,2019-07-01,93000
1,1,2019-07-01,63000
2,2,2019-07-01,6000
3,3,2019-07-01,57000
4,4,2019-07-01,0
...,...,...,...
4411,19,2019-12-31,226500
4412,20,2019-12-31,256500
4413,21,2019-12-31,262000
4414,22,2019-12-31,242500


In [34]:
df_date = pd.DataFrame(xlsx[['판매일자','판매시간','영수증번호','메뉴가격*수량']])
df_date

,판매일자,판매시간,영수증번호,메뉴가격*수량
0,2020-07-01,09:58:56,01-0001,14000
1,2020-07-01,09:58:56,01-0001,8000
2,2020-07-01,10:19:45,01-0002,16000
3,2020-07-01,10:19:45,01-0002,8000
4,2020-07-01,10:19:45,01-0002,12000
...,...,...,...,...
8589,2020-07-31,09:44:37,01-0192,8000
8590,2020-07-31,09:44:37,01-0192,8500
8591,2020-07-31,09:44:49,01-0193,14000
8592,2020-07-31,09:46:56,01-0194,20000


In [35]:
chk_dict={}

pos=0

pre_date=df_date.loc[0]['판매일자']

flag=False
temp_dict={}

for i in df_date.index:
    
    date=df_date.loc[i]['판매일자']
    
    if i>0:
        pre_time = df_date.iloc[i-1]['판매시간'].split(":")
        cur_time = df_date.iloc[i]['판매시간'].split(":")
        
        pre_hour = int(pre_time[0])
        cur_hour = int(cur_time[0])
        ## 이전행의 날짜와 현재행의 날짜가 같고 시간은 00시를 넘겼을 경우 flag=True
        if (pre_date==date) and (df_date.iloc[i-1]['판매시간']>df_date.iloc[i]['판매시간']):
            pos=i
            flag = True
        elif (pre_date!=date) or (i==(len(df_date)-1)):
            flag=False
            
            if i==(len(df_date)-1):
                change_date = dt.datetime.strptime(date, '%Y-%m-%d')
                print('date:',date,',','change_date:',str(change_date+ dt.timedelta(days=1))[:10])
                df_date.loc[i,'판매일자']=str(change_date + dt.timedelta(days=1))[:10]
                xlsx.loc[i,'판매일자']=str(change_date + dt.timedelta(days=1))[:10]
            
            chk_dict[pre_date]=temp_dict # 어긋난 날짜의 각 시간대별 매출액 저장
#             print("i:",i,",","pre_date:",pre_date)
#             print(temp_dict)
            temp_dict={}
        
        ## 00시가 넘었는데 날짜가 바뀌지 않은 데이터들(flag=True) 처리 
        if flag:
            print(i)
            
            ## 판매일자 00시 넘어가면 하루 더하기
            change_date = dt.datetime.strptime(date, '%Y-%m-%d')
            print('date:',date,',','change_date:',str(change_date+ dt.timedelta(days=1))[:10])
            df_date.loc[i,'판매일자']=str(change_date + dt.timedelta(days=1))[:10]
            xlsx.loc[i,'판매일자']=str(change_date + dt.timedelta(days=1))[:10]
            
            ## 시간대별로 매출액이 얼마나 발생했는지 저장
            if temp_dict.get(cur_hour):
                temp_dict[cur_hour]+=int(df_date.iloc[i]['메뉴가격*수량'])
            else:
                temp_dict[cur_hour]=int(df_date.iloc[i]['메뉴가격*수량'])
         
    pre_date = date

196
date: 2020-07-01 , change_date: 2020-07-02
197
date: 2020-07-01 , change_date: 2020-07-02
198
date: 2020-07-01 , change_date: 2020-07-02
199
date: 2020-07-01 , change_date: 2020-07-02
200
date: 2020-07-01 , change_date: 2020-07-02
201
date: 2020-07-01 , change_date: 2020-07-02
202
date: 2020-07-01 , change_date: 2020-07-02
203
date: 2020-07-01 , change_date: 2020-07-02
204
date: 2020-07-01 , change_date: 2020-07-02
205
date: 2020-07-01 , change_date: 2020-07-02
206
date: 2020-07-01 , change_date: 2020-07-02
207
date: 2020-07-01 , change_date: 2020-07-02
208
date: 2020-07-01 , change_date: 2020-07-02
209
date: 2020-07-01 , change_date: 2020-07-02
210
date: 2020-07-01 , change_date: 2020-07-02
211
date: 2020-07-01 , change_date: 2020-07-02
212
date: 2020-07-01 , change_date: 2020-07-02
213
date: 2020-07-01 , change_date: 2020-07-02
214
date: 2020-07-01 , change_date: 2020-07-02
215
date: 2020-07-01 , change_date: 2020-07-02
216
date: 2020-07-01 , change_date: 2020-07-02
217
date: 202

770
date: 2020-07-03 , change_date: 2020-07-04
771
date: 2020-07-03 , change_date: 2020-07-04
772
date: 2020-07-03 , change_date: 2020-07-04
773
date: 2020-07-03 , change_date: 2020-07-04
774
date: 2020-07-03 , change_date: 2020-07-04
775
date: 2020-07-03 , change_date: 2020-07-04
776
date: 2020-07-03 , change_date: 2020-07-04
777
date: 2020-07-03 , change_date: 2020-07-04
778
date: 2020-07-03 , change_date: 2020-07-04
779
date: 2020-07-03 , change_date: 2020-07-04
780
date: 2020-07-03 , change_date: 2020-07-04
781
date: 2020-07-03 , change_date: 2020-07-04
782
date: 2020-07-03 , change_date: 2020-07-04
783
date: 2020-07-03 , change_date: 2020-07-04
784
date: 2020-07-03 , change_date: 2020-07-04
785
date: 2020-07-03 , change_date: 2020-07-04
786
date: 2020-07-03 , change_date: 2020-07-04
787
date: 2020-07-03 , change_date: 2020-07-04
788
date: 2020-07-03 , change_date: 2020-07-04
789
date: 2020-07-03 , change_date: 2020-07-04
790
date: 2020-07-03 , change_date: 2020-07-04
791
date: 202

1771
date: 2020-07-07 , change_date: 2020-07-08
1772
date: 2020-07-07 , change_date: 2020-07-08
1773
date: 2020-07-07 , change_date: 2020-07-08
1774
date: 2020-07-07 , change_date: 2020-07-08
1775
date: 2020-07-07 , change_date: 2020-07-08
1776
date: 2020-07-07 , change_date: 2020-07-08
1777
date: 2020-07-07 , change_date: 2020-07-08
1778
date: 2020-07-07 , change_date: 2020-07-08
1779
date: 2020-07-07 , change_date: 2020-07-08
1780
date: 2020-07-07 , change_date: 2020-07-08
1781
date: 2020-07-07 , change_date: 2020-07-08
1782
date: 2020-07-07 , change_date: 2020-07-08
1783
date: 2020-07-07 , change_date: 2020-07-08
1784
date: 2020-07-07 , change_date: 2020-07-08
1785
date: 2020-07-07 , change_date: 2020-07-08
1786
date: 2020-07-07 , change_date: 2020-07-08
1787
date: 2020-07-07 , change_date: 2020-07-08
1788
date: 2020-07-07 , change_date: 2020-07-08
1789
date: 2020-07-07 , change_date: 2020-07-08
1790
date: 2020-07-07 , change_date: 2020-07-08
1791
date: 2020-07-07 , change_date: 202

2379
date: 2020-07-09 , change_date: 2020-07-10
2380
date: 2020-07-09 , change_date: 2020-07-10
2381
date: 2020-07-09 , change_date: 2020-07-10
2382
date: 2020-07-09 , change_date: 2020-07-10
2383
date: 2020-07-09 , change_date: 2020-07-10
2384
date: 2020-07-09 , change_date: 2020-07-10
2385
date: 2020-07-09 , change_date: 2020-07-10
2386
date: 2020-07-09 , change_date: 2020-07-10
2387
date: 2020-07-09 , change_date: 2020-07-10
2388
date: 2020-07-09 , change_date: 2020-07-10
2389
date: 2020-07-09 , change_date: 2020-07-10
2390
date: 2020-07-09 , change_date: 2020-07-10
2391
date: 2020-07-09 , change_date: 2020-07-10
2392
date: 2020-07-09 , change_date: 2020-07-10
2393
date: 2020-07-09 , change_date: 2020-07-10
2394
date: 2020-07-09 , change_date: 2020-07-10
2395
date: 2020-07-09 , change_date: 2020-07-10
2396
date: 2020-07-09 , change_date: 2020-07-10
2397
date: 2020-07-09 , change_date: 2020-07-10
2398
date: 2020-07-09 , change_date: 2020-07-10
2399
date: 2020-07-09 , change_date: 202

3023
date: 2020-07-11 , change_date: 2020-07-12
3024
date: 2020-07-11 , change_date: 2020-07-12
3025
date: 2020-07-11 , change_date: 2020-07-12
3026
date: 2020-07-11 , change_date: 2020-07-12
3027
date: 2020-07-11 , change_date: 2020-07-12
3028
date: 2020-07-11 , change_date: 2020-07-12
3029
date: 2020-07-11 , change_date: 2020-07-12
3030
date: 2020-07-11 , change_date: 2020-07-12
3031
date: 2020-07-11 , change_date: 2020-07-12
3032
date: 2020-07-11 , change_date: 2020-07-12
3033
date: 2020-07-11 , change_date: 2020-07-12
3034
date: 2020-07-11 , change_date: 2020-07-12
3035
date: 2020-07-11 , change_date: 2020-07-12
3036
date: 2020-07-11 , change_date: 2020-07-12
3305
date: 2020-07-12 , change_date: 2020-07-13
3306
date: 2020-07-12 , change_date: 2020-07-13
3307
date: 2020-07-12 , change_date: 2020-07-13
3308
date: 2020-07-12 , change_date: 2020-07-13
3309
date: 2020-07-12 , change_date: 2020-07-13
3310
date: 2020-07-12 , change_date: 2020-07-13
3311
date: 2020-07-12 , change_date: 202

...

date: 2020-07-21 , change_date: 2020-07-22
5876
date: 2020-07-22 , change_date: 2020-07-23
5877
date: 2020-07-22 , change_date: 2020-07-23
5878
date: 2020-07-22 , change_date: 2020-07-23
5879
date: 2020-07-22 , change_date: 2020-07-23
5880
date: 2020-07-22 , change_date: 2020-07-23
5881
date: 2020-07-22 , change_date: 2020-07-23
5882
date: 2020-07-22 , change_date: 2020-07-23
5883
date: 2020-07-22 , change_date: 2020-07-23
5884
date: 2020-07-22 , change_date: 2020-07-23
5885
date: 2020-07-22 , change_date: 2020-07-23
5886
date: 2020-07-22 , change_date: 2020-07-23
5887
date: 2020-07-22 , change_date: 2020-07-23
5888
date: 2020-07-22 , change_date: 2020-07-23
5889
date: 2020-07-22 , change_date: 2020-07-23
5890
date: 2020-07-22 , change_date: 2020-07-23
5891
date: 2020-07-22 , change_date: 2020-07-23
5892
date: 2020-07-22 , change_date: 2020-07-23
5893
date: 2020-07-22 , change_date: 2020-07-23
5894
date: 2020-07-22 , change_date: 2020-07-23
5895
date: 2020-07-22 , change_date: 2020-07-

6423
date: 2020-07-24 , change_date: 2020-07-25
6424
date: 2020-07-24 , change_date: 2020-07-25
6425
date: 2020-07-24 , change_date: 2020-07-25
6426
date: 2020-07-24 , change_date: 2020-07-25
6427
date: 2020-07-24 , change_date: 2020-07-25
6428
date: 2020-07-24 , change_date: 2020-07-25
6429
date: 2020-07-24 , change_date: 2020-07-25
6430
date: 2020-07-24 , change_date: 2020-07-25
6431
date: 2020-07-24 , change_date: 2020-07-25
6432
date: 2020-07-24 , change_date: 2020-07-25
6433
date: 2020-07-24 , change_date: 2020-07-25
6434
date: 2020-07-24 , change_date: 2020-07-25
6435
date: 2020-07-24 , change_date: 2020-07-25
6436
date: 2020-07-24 , change_date: 2020-07-25
6437
date: 2020-07-24 , change_date: 2020-07-25
6438
date: 2020-07-24 , change_date: 2020-07-25
6439
date: 2020-07-24 , change_date: 2020-07-25
6440
date: 2020-07-24 , change_date: 2020-07-25
6441
date: 2020-07-24 , change_date: 2020-07-25
6442
date: 2020-07-24 , change_date: 2020-07-25
6443
date: 2020-07-24 , change_date: 202

7078
date: 2020-07-26 , change_date: 2020-07-27
7079
date: 2020-07-26 , change_date: 2020-07-27
7080
date: 2020-07-26 , change_date: 2020-07-27
7081
date: 2020-07-26 , change_date: 2020-07-27
7082
date: 2020-07-26 , change_date: 2020-07-27
7083
date: 2020-07-26 , change_date: 2020-07-27
7084
date: 2020-07-26 , change_date: 2020-07-27
7085
date: 2020-07-26 , change_date: 2020-07-27
7086
date: 2020-07-26 , change_date: 2020-07-27
7087
date: 2020-07-26 , change_date: 2020-07-27
7088
date: 2020-07-26 , change_date: 2020-07-27
7089
date: 2020-07-26 , change_date: 2020-07-27
7090
date: 2020-07-26 , change_date: 2020-07-27
7091
date: 2020-07-26 , change_date: 2020-07-27
7092
date: 2020-07-26 , change_date: 2020-07-27
7093
date: 2020-07-26 , change_date: 2020-07-27
7094
date: 2020-07-26 , change_date: 2020-07-27
7095
date: 2020-07-26 , change_date: 2020-07-27
7096
date: 2020-07-26 , change_date: 2020-07-27
7097
date: 2020-07-26 , change_date: 2020-07-27
7098
date: 2020-07-26 , change_date: 202

7657
date: 2020-07-28 , change_date: 2020-07-29
7658
date: 2020-07-28 , change_date: 2020-07-29
7659
date: 2020-07-28 , change_date: 2020-07-29
7660
date: 2020-07-28 , change_date: 2020-07-29
7661
date: 2020-07-28 , change_date: 2020-07-29
7662
date: 2020-07-28 , change_date: 2020-07-29
7663
date: 2020-07-28 , change_date: 2020-07-29
7664
date: 2020-07-28 , change_date: 2020-07-29
7665
date: 2020-07-28 , change_date: 2020-07-29
7666
date: 2020-07-28 , change_date: 2020-07-29
7667
date: 2020-07-28 , change_date: 2020-07-29
7668
date: 2020-07-28 , change_date: 2020-07-29
7669
date: 2020-07-28 , change_date: 2020-07-29
7670
date: 2020-07-28 , change_date: 2020-07-29
7879
date: 2020-07-29 , change_date: 2020-07-30
7880
date: 2020-07-29 , change_date: 2020-07-30
7881
date: 2020-07-29 , change_date: 2020-07-30
7882
date: 2020-07-29 , change_date: 2020-07-30
7883
date: 2020-07-29 , change_date: 2020-07-30
7884
date: 2020-07-29 , change_date: 2020-07-30
7885
date: 2020-07-29 , change_date: 202

8460
date: 2020-07-31 , change_date: 2020-08-01
8461
date: 2020-07-31 , change_date: 2020-08-01
8462
date: 2020-07-31 , change_date: 2020-08-01
8463
date: 2020-07-31 , change_date: 2020-08-01
8464
date: 2020-07-31 , change_date: 2020-08-01
8465
date: 2020-07-31 , change_date: 2020-08-01
8466
date: 2020-07-31 , change_date: 2020-08-01
8467
date: 2020-07-31 , change_date: 2020-08-01
8468
date: 2020-07-31 , change_date: 2020-08-01
8469
date: 2020-07-31 , change_date: 2020-08-01
8470
date: 2020-07-31 , change_date: 2020-08-01
8471
date: 2020-07-31 , change_date: 2020-08-01
8472
date: 2020-07-31 , change_date: 2020-08-01
8473
date: 2020-07-31 , change_date: 2020-08-01
8474
date: 2020-07-31 , change_date: 2020-08-01
8475
date: 2020-07-31 , change_date: 2020-08-01
8476
date: 2020-07-31 , change_date: 2020-08-01
8477
date: 2020-07-31 , change_date: 2020-08-01
8478
date: 2020-07-31 , change_date: 2020-08-01
8479
date: 2020-07-31 , change_date: 2020-08-01
8480
date: 2020-07-31 , change_date: 202

In [36]:
current = dt.datetime.strptime('2018-05-20','%Y-%m-%d')
res = current+dt.timedelta(days=1)
print(str(res)[:10])

2018-05-21


In [37]:
chk_dict

{'2020-07-01': {0: 64000,
  1: 38000,
  2: 15000,
  3: 56000,
  4: 173000,
  5: 76000,
  6: 8000,
  7: 97000,
  8: 91000,
  9: 107000},
 '2020-07-02': {0: 60000,
  1: 43500,
  2: 93000,
  3: 47000,
  4: 142500,
  5: 71500,
  6: 54000,
  7: 102500,
  8: 95500,
  9: 102500,
  10: 15500},
 '2020-07-03': {0: 120500,
  1: 96000,
  2: 153000,
  3: 122500,
  4: 156500,
  5: 142500,
  6: 207000,
  7: 219000,
  8: 129000,
  9: 148000},
 '2020-07-04': {0: 126500,
  1: 30500,
  2: 64000,
  3: 50000,
  4: 157500,
  5: 94000,
  6: 159000,
  7: 116000,
  8: 346000,
  9: 313000},
 '2020-07-05': {0: 113000,
  1: 74000,
  2: 39500,
  3: 39000,
  4: 95000,
  5: 53500,
  6: 73000,
  7: 116000,
  8: 45500,
  9: 68000},
 '2020-07-06': {},
 '2020-07-07': {0: 122500,
  1: 28000,
  2: 110500,
  3: 73000,
  4: 60500,
  5: 24000,
  6: 54500,
  7: 46500,
  8: 44500,
  9: 14000},
 '2020-07-08': {0: 92500,
  1: 80000,
  2: 72500,
  3: 56000,
  4: 115000,
  5: 59500,
  6: 117000,
  7: 28000,
  8: 60500,
  9: 13000}

In [38]:
df_date.loc[188]

판매일자       2020-07-01
판매시간         22:59:54
영수증번호         01-0115
메뉴가격*수량          1500
Name: 188, dtype: object

In [39]:
## 시간대별 매출 연쇄적으로 가격 변동시키기

for date in chk_dict:
    if len(chk_dict[date].keys())!=0:
        for time,price in chk_dict[date].items():
            pre_price=df_time_price[(df_time_price['날짜']==date)&(df_time_price['시간']==time)]['매출액']
            data_idx = df_time_price[(df_time_price['날짜']==date)&(df_time_price['시간']==time)].index
            print('date{0},time{1}:'.format(date,time),"data_idx:",df_time_price.loc[data_idx])
            df_time_price.loc[data_idx,'매출액']=pre_price-price
            
            if data_idx<(len(df_time_price)-24):
                change_date = dt.datetime.strptime(date, '%Y-%m-%d')
                next_date = str(change_date + dt.timedelta(days=1))[:10]
                next_price = df_time_price[(df_time_price['날짜']==next_date)&(df_time_price['시간']==time)]['매출액']
                df_time_price.loc[data_idx+24,'매출액']=next_price+price

date2020-07-01,time0: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time1: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time2: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time3: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time4: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time7: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time8: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-01,time9: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-02,time0: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-02,time1: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-02,time2: data_i

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value o

 data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-04,time4: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-04,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-04,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-04,time7: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-04,time8: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-04,time9: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-05,time0: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-05,time1: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-05,time2: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-05,time3: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-05,time4: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-05,time5: data_idx: Empty DataFrame
C

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value o

date2020-07-08,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-08,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-08,time7: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-08,time8: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-08,time9: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time0: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time1: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time2: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time3: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time4: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-09,time7: data_i

...

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value o

date2020-07-24,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-24,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-24,time7: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-24,time8: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-24,time9: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time0: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time1: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time2: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time3: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time4: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-25,time7: data_i

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value o

 data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-26,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-26,time7: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-26,time8: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-26,time9: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time0: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time1: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time2: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time3: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time4: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time5: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time6: data_idx: Empty DataFrame
Columns: [시간, 날짜, 매출액]
Index: []
date2020-07-27,time7: data_idx: Empty DataFrame
C

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The truth value o

In [40]:
print(len(chk_dict['2019-07-02'].keys()))

KeyError: '2019-07-02'

In [41]:
xlsx

,Unnamed: 0,판매일자,영수증번호,판매시간,영수증별총매출,현금,카드,key,구분,판매시간.1,메뉴코드,영수증번호.1,메뉴명,메뉴가격,메뉴수량,메뉴가격*수량
0,0,2020-07-01,01-0001,09:58:56,22000,0,22000,r1,판매,09:58:56,Z001,01-0001,돼지국밥,7000,2,14000
1,1,2020-07-01,01-0001,09:58:56,22000,0,22000,r1,판매,09:58:56,Z002,01-0001,순대국밥,8000,1,8000
2,2,2020-07-01,01-0002,10:19:45,36000,0,36000,r2,판매,10:19:45,Z031,01-0002,양념주물럭(1인분),8000,2,16000
3,3,2020-07-01,01-0002,10:19:45,36000,0,36000,r2,판매,10:19:45,Z054,01-0002,짬뽕순두부국밥,8000,1,8000
4,4,2020-07-01,01-0002,10:19:45,36000,0,36000,r2,판매,10:19:45,Z072,01-0002,참소주,4000,3,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,8589,2020-08-01,01-0192,09:44:37,23500,0,23500,r4852,판매,09:44:37,Z002,01-0192,순대국밥,8000,1,8000
8590,8590,2020-08-01,01-0192,09:44:37,23500,0,23500,r4852,판매,09:44:37,Z005,01-0192,모둠국밥,8500,1,8500
8591,8591,2020-08-01,01-0193,09:44:49,14000,14000,0,r4853,판매,09:44:49,Z001,01-0193,돼지국밥,7000,2,14000
8592,8592,2020-08-01,01-0194,09:46:56,32000,0,32000,r4854,판매,09:46:56,Z038,01-0194,무보까정식,10000,2,20000


In [42]:
df_time_price

,시간,날짜,매출액
0,0,2019-07-01,93000
1,1,2019-07-01,63000
2,2,2019-07-01,6000
3,3,2019-07-01,57000
4,4,2019-07-01,0
...,...,...,...
4411,19,2019-12-31,226500
4412,20,2019-12-31,256500
4413,21,2019-12-31,262000
4414,22,2019-12-31,242500


In [43]:
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os

f_dir = f_dir+"\\\\"
# 저장될 파일위치와 이름을 지정합니다
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+keyword)
os.chdir(f_dir+s+'-'+keyword)

fc_name=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.csv'

xlsx.to_csv(fc_name,encoding="utf-8-sig",index=False)

In [ ]:
os.makedirs(f_dir+s+'-'+time_keyword)
os.chdir(f_dir+s+'-'+time_keyword)

fc_name=f_dir+s+'-'+time_keyword+'\\'+s+'-'+time_keyword+'.csv'

df_time_price.to_csv(fc_name,encoding="utf-8-sig",index=False)